In [1]:
import pandas as pd
import numpy as np

In [2]:
pop1 = pd.read_csv('/Users/caleboneel/Desktop/estimating-impact-of-opioids-2020-purlple-team/20_intermediate_files/countypopulations_clean.csv')

vitality = pd.read_csv('/Users/caleboneel/Desktop/estimating-impact-of-opioids-2020-purlple-team/10_code/od_deaths_state.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/Users/caleboneel/Desktop/estimating-impact-of-opioids-2020-purlple-team/20_intermediate_files/countypopulations_clean.csv'

In [ ]:
mergepop = pop1.copy()
mergepop = mergepop.rename(columns = {'FIPS':'County Code', 'State Abbr':'State'}) 
mergepop.drop(['State Name','Unnamed: 0','_merge'], axis=1, inplace = True)
mergepop.drop(mergepop.loc[mergepop['Year']== 2000.0].index, inplace=True)
mergepop.drop(mergepop.loc[mergepop['Year']== 2001.0].index, inplace=True)
mergepop.drop(mergepop.loc[mergepop['Year']== 2002.0].index, inplace=True)
mergepop.drop(mergepop.loc[mergepop['Year']== 2016.0].index, inplace=True)
mergepop.drop(mergepop.loc[mergepop['Year']== 2017.0].index, inplace=True)
mergepop.drop(mergepop.loc[mergepop['Year']== 2018.0].index, inplace=True)
mergepop.drop(mergepop.loc[mergepop['Year']== 2019.0].index, inplace=True)
mergepop.drop(mergepop.loc[mergepop['State']== 'ND'].index, inplace=True)
mergepop

In [ ]:
vitality

In [ ]:
combo = vitality.merge(mergepop, how = 'right', on = ['County Code', 'Year'], indicator = True)

In [ ]:
combo

In [ ]:
death_coef = combo[combo['_merge'] == 'both']

In [ ]:
death_coef = death_coef[['State_x', 'Year', 'Deaths', 'Population']]

In [ ]:
coef = death_coef.groupby(['State_x', 'Year'], as_index = False).sum()

In [ ]:
coef['death rate'] = coef['Deaths']/coef['Population']
coef.rename(columns = {'State_x':'State'}, inplace = True) 
coef

# Markdown incase reuse
wyoming = coef[coef['State_x'] == 'WY']
wyoming

wy = pop1.groupby(['State Abbr', 'Year'], as_index = False).sum()
wy1 = wy[wy['State Abbr'] == 'WY']
wy1

In [ ]:
deathrate = coef.drop(['Deaths', 'Population'], axis=1)

In [ ]:
combo['Merge?'] = combo['_merge']
combo.drop(['_merge'], axis=1, inplace = True)

In [ ]:
combo.rename(columns = {'State_y':'State', 'County_y':'County'}, inplace = True)
combo.drop(['County_x', 'State_x'], axis = 1, inplace = True)

In [ ]:
death = combo.merge(deathrate, how='left', on = ['State', 'Year'], indicator = True)

In [ ]:
death['death rate'].isna().sum()
deathnan = death[death['_merge'] == 'left_only']

multiple = death.copy()
if ((multiple['Merge?'] == 'right_only') and (multiple['_merge'] == 'both')):
    deathcoef = round(multiple['Population'] * multiple['death rate'])
    multiple['Deaths'] = min(9, deathcoef)

multiple = death.copy()
multiple['deathcoef'] = round(multiple['Population'] * multiple['death rate'])
multiple.loc[multiple['Merge?'] == 'right_only', 'projdeaths'] = min(9, deathcoef)

In [ ]:
deathnanyr = deathnan.groupby(['State','Year'], as_index = False).sum()
deathnanyr

In [ ]:
deathnanyr['State'].value_counts()


In [ ]:
multiple = death.copy()
missing =  multiple[multiple['State'].isin(['SD', 'VT', 'WY', 'NE', 'MT'])]  

In [ ]:
SD = missing[missing['State']== 'SD']
VT = missing[missing['State']== 'VT']
WY = missing[missing['State']== 'WY']
NE = missing[missing['State']== 'NE']
MT = missing[missing['State']== 'MT']

In [ ]:
SD = SD.groupby(['Year'], as_index = False).mean()
SDrate = SD['death rate'].mean()

VT = VT.groupby(['Year'], as_index = False).mean()
VTrate = VT['death rate'].mean()

WY = WY.groupby(['Year'], as_index = False).mean()
WYrate = WY['death rate'].mean()

NE = NE.groupby(['Year'], as_index = False).mean()
NErate = NE['death rate'].mean()

MT = MT.groupby(['Year'], as_index = False).mean()
MTrate = MT['death rate'].mean()

In [ ]:
multcopy = multiple.copy()
multcopy

In [ ]:
multcopy['death rate'] = np.where((multcopy.State == 'SD')&(multcopy['death rate'].isnull()),
                                 SDrate,
                                 multcopy['death rate'])

In [ ]:
multcopy['death rate'] = np.where((multcopy.State == 'VT')&(multcopy['death rate'].isnull()),
                                 VTrate,
                                 multcopy['death rate'])

In [ ]:
multcopy['death rate'] = np.where((multcopy.State == 'WY')&(multcopy['death rate'].isnull()),
                                 WYrate,
                                 multcopy['death rate'])

In [ ]:
multcopy['death rate'] = np.where((multcopy.State == 'NE')&(multcopy['death rate'].isnull()),
                                 NErate,
                                 multcopy['death rate'])

In [ ]:
multcopy['death rate'] = np.where((multcopy.State == 'MT')&(multcopy['death rate'].isnull()),
                                 MTrate,
                                 multcopy['death rate'])

In [ ]:
SDnew = multcopy[multcopy['State'] == 'SD']

In [ ]:
SDnew.groupby(['Year']).mean()

In [ ]:
VTnew = multcopy[multcopy['State'] == 'VT']
VTnew

In [ ]:
multcopy['death rate'].isnull().values.any()
multcopy

In [ ]:
#multcopy.drop(['deathcoef'], axis=1, inplace = True)

In [ ]:
multcopy['deathcoef'] = round(multcopy['Population'] * multcopy['death rate'])


In [ ]:
multcopy['projdeaths'] = np.where((multcopy.deathcoef < 9),
                                 multcopy.deathcoef,
                                 9)

In [ ]:
multcopy

In [ ]:
multcopy['CalculatedDeaths'] = np.where((multcopy.Deaths.isnull()),
                                 multcopy.projdeaths,
                                 multcopy.Deaths)

In [ ]:
assert len(multcopy) == len(mergepop)
assert multcopy.projdeaths.isnull().values.any() == False

In [ ]:
multcopy.drop(['Merge?', '_merge'], axis=1, inplace = True)

In [ ]:
vitalitydf = multcopy.copy()

In [ ]:
vitalitydf.to_csv('vitalitydf_allcounties', index=False)